In [1]:
# Import necessary libraries

from splinter import Browser
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd


In [2]:
def init_browser():
    
    executable_path ={'executable_path': 'chromedriver'}
    return Browser('chrome', **executable_path, headless=False)

In [3]:
# Sec A: Scraping
# Complete your initial scraping using Jupyter Notebook, BeautifulSoup, Pandas, and Requests/Splinter.

In [4]:
def scrape():
    browser = init_browser()
    listings = {}
    # NASA Mars News
    url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
    browser.visit(url)
    html = browser.html
    soup = bs(html, 'html.parser')
    news_title = soup.find('div', class_="content_title").text
    news_text = soup.find('div', class_="article_teaser_body").text
    
    # 2. JPL Mars Space Images - Featured Image
    url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
    browser.visit(url)
    html = browser.html
    soup = bs(html, 'html.parser')
    imgbody = soup.find("a", class_="fancybox").get('data-fancybox-href').strip()
    imgurl = 'https://www.jpl.nasa.gov'+imgbody
    
    # 3. Mars Weather
    url = 'https://twitter.com/marswxreport?lang=en'
    browser.visit(url)
    html = browser.html
    soup = bs(html, 'html.parser')
    tweet = soup.find(class_="js-tweet-text").text
    weather = tweet.split('pic')[0] 
    
    # 4. Mars Facts
    url = 'https://space-facts.com/mars/'
    tables = pd.read_html(url)
    df = tables[0]
    fact_list=df.values.tolist()
    
    # 5. Mars Hem
    url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
    browser.visit(url)
    html = browser.html
    soup = bs(html, 'html.parser')
    url_list = []
    itemlinks = soup.find_all(class_="description")

    for itemlink in itemlinks:
        item = itemlink.find('a', class_="itemLink product-item").get('href')
        url_list.append(item)
    item_url_list = ['https://astrogeology.usgs.gov/' + item_url for item_url in url_list]
    
    title_list = []
    image_urls_list = []
    for x in range (0,4):

        item_url = item_url_list[x]
        browser.visit(item_url)
        html = browser.html
        soup = bs(html, 'html.parser')
    #     print(item_url)
        image_urls = soup.find("li").find('a')['href']
        title = soup.find('h2', class_="title").text
        title_list.append(title)
        image_urls_list.append(image_urls)
    hemisphere_image_urls = [{"title": title_list[0], "img_url": image_urls_list[0]},
                            {"title": title_list[1], "img_url": image_urls_list[1]},
                            {"title": title_list[2], "img_url": image_urls_list[2]},
                            {"title": title_list[3], "img_url": image_urls_list[3]}]
    listings = [{'news':news_title}, 
           {'text':news_text}, 
           {'url':imgurl}, 
           {'weather':weather},
           {'description':fact_list[0][0], 'value':fact_list[0][1]},
           {'description':fact_list[1][0], 'value':fact_list[1][1]},
           {'description':fact_list[2][0], 'value':fact_list[2][1]},
           {'description':fact_list[3][0], 'value':fact_list[3][1]},
           {'description':fact_list[4][0], 'value':fact_list[4][1]},
           {'description':fact_list[5][0], 'value':fact_list[5][1]},
           {'description':fact_list[6][0], 'value':fact_list[6][1]},
           {'description':fact_list[7][0], 'value':fact_list[7][1]},
           {'description':fact_list[8][0], 'value':fact_list[8][1]},
           {"title": title_list[0], "img_url": image_urls_list[0]},                       
           {"title": title_list[1], "img_url": image_urls_list[1]},
           {"title": title_list[2], "img_url": image_urls_list[2]},
           {"title": title_list[3], "img_url": image_urls_list[3]},
           ]
    return listings
    print(listings)

In [5]:
import pymongo

conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.testmars_db
collection = db.items
collection.insert_many(listings)

NameError: name 'listings' is not defined

In [ ]:
    print(listings)

In [12]:
import pymongo
testlist =[{'_id': ('5df00189170f74023ec3742b'), 'news': 'Two Rovers to Roll on Mars Again: Curiosity and Mars 2020'}, {'_id': ('5df00189170f74023ec3742c'), 'text': 'They look like twins. But under the hood, the rover currently exploring the Red Planet and the one launching there this summer have distinct science tools and roles to play.'}, {'_id': ('5df00189170f74023ec3742d'), 'url': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA17462_ip.jpg'}, {'_id': ('5df00189170f74023ec3742e'), 'weather': 'InSight sol 367 (2019-12-08) low -139.9ºC (-219.9ºF) high -20.9ºC (-5.6ºF)\nwinds from the SSE at 5.9 m/s (13.3 mph) gusting to 21.9 m/s (49.0 mph)\npressure at 6.60 hPa'}, {'_id': ('5df00189170f74023ec3742f'), 'description': 'Equatorial Diameter:', 'value': '6,792 km'}, {'_id': ('5df00189170f74023ec37430'), 'description': 'Polar Diameter:', 'value': '6,752 km'}, {'_id': ('5df00189170f74023ec37431'), 'description': 'Mass:', 'value': '6.39 × 10^23 kg (0.11 Earths)'}, {'_id': ('5df00189170f74023ec37432'), 'description': 'Moons:', 'value': 
'2 (Phobos & Deimos)'}, {'_id': ('5df00189170f74023ec37433'), 'description': 'Orbit Distance:', 'value': '227,943,824 km (1.38 AU)'}, {'_id': ('5df00189170f74023ec37434'), 'description': 'Orbit Period:', 'value': '687 days (1.9 years)'}, {'_id': ('5df00189170f74023ec37435'), 'description': 'Surface Temperature:', 'value': '-87 to -5 °C'}, {'_id': ('5df00189170f74023ec37436'), 'description': 'First Record:', 'value': '2nd millennium BC'}, {'_id': ('5df00189170f74023ec37437'), 'description': 'Recorded By:', 'value': 'Egyptian astronomers'}, {'_id': ('5df00189170f74023ec37438'), 'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'_id': ('5df00189170f74023ec37439'), 'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'_id': ('5df00189170f74023ec3743a'), 'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'_id': ('5df00189170f74023ec3743b'), 'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


In [15]:
testlist.text

AttributeError: 'list' object has no attribute 'text'

In [19]:
a=testlist[0].get("news")
print(a)

Two Rovers to Roll on Mars Again: Curiosity and Mars 2020


In [22]:
a=testlist[1].get("text")
print(a)

They look like twins. But under the hood, the rover currently exploring the Red Planet and the one launching there this summer have distinct science tools and roles to play.
